<a href="https://colab.research.google.com/github/TheAllBlueChevy/UST-Deep-Learning-2026/blob/main/HWK5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HWK5**

# 0) Importing Libraries

In [49]:
import time
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

np.random.seed(42)
tf.random.set_seed(42)

# 1) Data Set Up

In [50]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1QbOSExVJEbPMhjzaua5n2eIXeF3qELQ7' -O "label.txt"

--2026-02-09 19:11:48--  https://drive.google.com/uc?export=download&id=1QbOSExVJEbPMhjzaua5n2eIXeF3qELQ7
Resolving drive.google.com (drive.google.com)... 142.250.4.139, 142.250.4.138, 142.250.4.113, ...
Connecting to drive.google.com (drive.google.com)|142.250.4.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1QbOSExVJEbPMhjzaua5n2eIXeF3qELQ7&export=download [following]
--2026-02-09 19:11:48--  https://drive.usercontent.google.com/download?id=1QbOSExVJEbPMhjzaua5n2eIXeF3qELQ7&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.217.194.132, 2404:6800:4003:c04::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.217.194.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60000 (59K) [application/octet-stream]
Saving to: ‘label.txt’

label.txt           100%[===================>]  58.59K  --.-KB/s    in 0.0

In [51]:
!pip install gdown
!gdown --id '1Sh2ce0jo5FVGNsSa9fqLjqcAOWQBFhzz' -O "encoded_seq.txt"

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:139: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1Sh2ce0jo5FVGNsSa9fqLjqcAOWQBFhzz
From (redirected): https://drive.google.com/uc?id=1Sh2ce0jo5FVGNsSa9fqLjqcAOWQBFhzz&confirm=t&uuid=24765eaf-e727-4a0a-84d8-a4cc93c44563
To: /content/encoded_seq.txt
100% 192M/192M [00:01<00:00, 156MB/s]


In [52]:
Length = 400

def load_data():

    labels = np.loadtxt('label.txt')
    encoded_seq = np.loadtxt('encoded_seq.txt')
    encoded_seq_choose = encoded_seq[:, ((400-Length)*2):(1600-(400-Length)*2)]

    print(encoded_seq_choose.shape)
    x_train,x_test,y_train,y_test = train_test_split(encoded_seq_choose,labels,test_size=0.2)

    return np.array(x_train),np.array(y_train),np.array(x_test),np.array(y_test)


x_train,y_train,x_test,y_test = load_data()

(30000, 1600)


In [53]:
print(y_train.shape)
print(y_train[:5])

(24000,)
[2. 0. 2. 0. 1.]


# 2) Model Builder

In [54]:
def cnn_model():
    model = Sequential([
        Reshape((400, 4), input_shape=(1600,)),
        Conv1D(
            filters=50,
            kernel_size=9,
            activation="relu",
            input_shape=(400, 4)
        ),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(3, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [55]:
model = cnn_model()
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_1 (Reshape)             │ (None, 400, 4)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 392, 50)        │         1,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 19600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │     1,960,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,962,253 (7.49 MB)

 Trainable params: 1,962,253 (7.49 MB)

 Non-trainable params: 0 (0.00 B)

In [56]:
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

# 3) Training on CPU/GPU

In [57]:
tf.keras.backend.clear_session()

with tf.device("/CPU:0"):
    cpu_model = cnn_model()

    start_time = time.time()
    cpu_model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        epochs=50,
        batch_size=32,
        callbacks=[early_stop],
        verbose=1
    )

    cpu_time = time.time() - start_time

Epoch 1/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 27s 35ms/step - accuracy: 0.8192 - loss: 0.4432 - val_accuracy: 0.9693 - val_loss: 0.1006
Epoch 2/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 19s 26ms/step - accuracy: 0.9730 - loss: 0.0887 - val_accuracy: 0.9677 - val_loss: 0.1071
Epoch 3/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 27ms/step - accuracy: 0.9868 - loss: 0.0452 - val_accuracy: 0.9663 - val_loss: 0.1128
Epoch 4/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.9931 - loss: 0.0257 - val_accuracy: 0.9708 - val_loss: 0.1108


In [58]:
tf.keras.backend.clear_session()
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [59]:
gpu_time = None

if tf.config.list_physical_devices("GPU"):
    with tf.device("/GPU:0"):
        gpu_model = cnn_model()

        start_time = time.time()
        gpu_model.fit(
            x_train, y_train,
            validation_data=(x_test, y_test),
            epochs=50,
            batch_size=32,
            callbacks=[early_stop],
            verbose=1
        )
        gpu_time = time.time() - start_time
else:
    print("No GPU detected.")


Epoch 1/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8174 - loss: 0.4963 - val_accuracy: 0.9698 - val_loss: 0.1046
Epoch 2/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9727 - loss: 0.0933 - val_accuracy: 0.9652 - val_loss: 0.1148
Epoch 3/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9854 - loss: 0.0519 - val_accuracy: 0.9682 - val_loss: 0.1217
Epoch 4/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9935 - loss: 0.0261 - val_accuracy: 0.9655 - val_loss: 0.1375


# 4) Results

In [60]:
print("\nTraining Time Comparison")
print(f"Craigs CPU Time: {cpu_time:.2f} seconds")

if gpu_time is not None:
    print(f"Craigs GPU Time: {gpu_time:.2f} seconds")
    print(f"Faster By: {cpu_time / gpu_time:.2f}×")



Training Time Comparison
Craigs CPU Time: 87.29 seconds
Craigs GPU Time: 16.11 seconds
Faster By: 5.42×
